# TNT Loop-Bridge: Stress Relaxation
> **Handbook:** This notebook demonstrates the TNT Loop-Bridge model. For complete mathematical derivations and theoretical background, see [TNT Loop-Bridge Documentation](../../docs/source/models/tnt/tnt_loop_bridge.rst).


**Estimated Time:** 3-5 minutes

## Protocol: Stress Relaxation in Loop-Bridge Networks

**Stress relaxation** in loop-bridge systems exhibits **two-exponential decay** from distinct loop and bridge relaxation timescales.

### Physical Context

After step strain:
1. **Fast mode** ($\tau_{loop}$): Intra-chain loop relaxation (stress-free equilibration)
2. **Slow mode** ($\tau_{bridge}$): Inter-chain bridge breakage (stress-bearing)

Relaxation modulus:

$$
G(t) = G_{bridge} \exp(-t/\tau_{bridge}) + G_{loop} \exp(-t/\tau_{loop})
$$

Note: At t=0⁺, both populations contribute to modulus; at long times, only bridge stress decays to zero.

> **Loop-Bridge Relaxation Physics**  
> For two-timescale analysis and population dynamics, see:  
> [../../docs/source/models/tnt/tnt_loop_bridge.rst](../../docs/source/models/tnt/tnt_loop_bridge.rst)

---

## Learning Objectives

1. Understand two-species relaxation in loop-bridge networks
2. Fit TNT Loop-Bridge model to $G(t)$ data
3. Extract loop and bridge timescales from bi-exponential decay
4. Analyze loop fraction from relative amplitudes
5. Compare loop-bridge to Cates (also bi-exponential but different physics)
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- TNT Loop-Bridge fundamentals (Notebook 13)
- Understanding of stress relaxation (TNT Single-Mode NB03)

## Runtime Estimate

- NLSQ fitting: ~5-10 seconds
- Bayesian inference: ~60-120 seconds
- Total: ~5-8 minutes

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import matplotlib.pyplot as plt
import numpy as np

from rheojax.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64

verify_float64()

from rheojax.models.tnt import TNTLoopBridge

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    compute_bell_effective_lifetime,
    compute_fit_quality,
    compute_maxwell_moduli,
    get_tnt_loop_bridge_param_names,
    load_epstein_saos,
    load_laponite_relaxation,
    load_ml_ikh_creep,
    load_ml_ikh_flow_curve,
    load_pnas_laos,
    load_pnas_startup,
    plot_bell_nu_sweep,
    plot_loop_bridge_fraction,
    print_convergence_summary,
    print_nu_interpretation,
    print_parameter_comparison,
    save_tnt_results,
)

param_names = get_tnt_loop_bridge_param_names()

from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## Theory: Stress Relaxation Dynamics

### Physical Picture

After flow cessation (gamma_dot → 0):
1. **Initial state**: Bridges depleted (f_B < f_B_eq) from prior shear
2. **Early times**: Elastic stress relaxes via tau_b (bridge detachment without force)
3. **Intermediate**: Bridges re-attach (loops → bridges)
4. **Late times**: Bridge fraction recovers to f_B_eq

### Governing Equations

**Bridge Fraction Recovery (no shear, gamma_dot = 0):**
```
df_B/dt = (1 - f_B)/tau_a - f_B/tau_b
```

**Steady State (equilibrium):**
```
f_B_eq = 1 / (1 + tau_a/tau_b)
```

**Solution (exponential recovery):**
```
f_B(t) = f_B_eq + (f_B_0 - f_B_eq) * exp(-t / tau_eq)
tau_eq = tau_a * tau_b / (tau_a + tau_b)
```

**Stress Relaxation (Maxwell backbone):**
```
G(t) = f_B(t) * G * exp(-t/tau_b)
```

### Two-Timescale Relaxation

1. **Fast relaxation (tau_b)**: Elastic stress decay via bridge detachment
2. **Slow recovery (tau_a)**: Modulus recovery via bridge re-attachment

If tau_a >> tau_b:
- Fast elastic relaxation followed by slow structural recovery
- G(t) decays quickly, then partially recovers

If tau_a ~ tau_b:
- Coupled relaxation-recovery, single effective timescale tau_eq

### Ratio Effects

- **tau_a/tau_b >> 1**: Slow attachment, low f_B_eq, weak recovery
- **tau_a/tau_b ~ 1**: Balanced kinetics, moderate f_B_eq
- **tau_a/tau_b << 1**: Fast attachment, high f_B_eq, strong recovery

## Load Relaxation Data

In [ ]:
time_data, modulus = load_laponite_relaxation(aging_time=1800)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} - {time_data.max():.2e} s")
print(f"Modulus range: {modulus.min():.2e} - {modulus.max():.2e} Pa")
print(f"Aging time: 1800 s")

fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(time_data, modulus, 'o', label='Data', markersize=6)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Relaxation Modulus G(t) (Pa)', fontsize=12)
ax.set_title('Stress Relaxation Data', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close("all")
plt.close('all')

## NLSQ Fitting

In [ ]:
# CI mode: Skip slow NLSQ fit - use reasonable defaults
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

model = TNTLoopBridge()

# Pre-shear rate used before relaxation starts
gamma_dot = 10.0  # s⁻¹ (typical pre-shear rate)

if FAST_MODE:
    print("FAST_MODE: Using default parameters (NLSQ fit takes >300s)")
    # Set reasonable parameters for relaxation
    model.parameters.set_value('G', 1000.0)  # Pa
    model.parameters.set_value('tau_b', 100.0)  # s (relaxation time from data)
    model.parameters.set_value('tau_a', 500.0)  # s
    model.parameters.set_value('f_B_eq', 0.5)
    model.parameters.set_value('eta_s', 0.01)  # Pa·s
    t_nlsq = 0.0
else:
    print("Starting NLSQ fit...")
    t_start = time.time()
    nlsq_result = model.fit(time_data, modulus, test_mode='relaxation', gamma_dot=gamma_dot, method='scipy')
    t_nlsq = time.time() - t_start
    print(f"\nNLSQ fit completed in {t_nlsq:.2f} seconds")

print(f"\nFitted parameters:")
for name in param_names:
    value = model.parameters.get_value(name)
    print(f"  {name}: {value:.4e}")

modulus_pred_fit = model.predict(time_data, test_mode='relaxation', gamma_dot=gamma_dot)
metrics = compute_fit_quality(modulus, modulus_pred_fit)
print(f"\nFit quality:")
print(f"  R²: {metrics['R2']:.6f}")
print(f"  RMSE: {metrics['RMSE']:.4e}")


## NLSQ Fit Visualization

In [ ]:
# Compute metrics for plot title
G_t = modulus  # Alias for consistency
metrics = compute_fit_quality(modulus, model.predict(time_data, test_mode='relaxation', gamma_dot=gamma_dot))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    time_data, modulus, model, test_mode="relaxation",
    param_names=param_names, log_scale=True,
    xlabel='Time (s)',
    ylabel=r'Relaxation modulus $G(t)$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})',
    gamma_dot=gamma_dot
)
plt.close("all")

## Physical Analysis: Bridge Re-equilibration

In [ ]:
# Generate fine grid for smooth predictions
time_pred = np.linspace(time_data.min(), time_data.max(), 200)
modulus_pred = model.predict(time_pred, test_mode='relaxation', gamma_dot=gamma_dot)

# Compute equilibrium and re-equilibration timescale
tau_eq = model.parameters.get_value('tau_a') * model.parameters.get_value('tau_b') / (model.parameters.get_value('tau_a') + model.parameters.get_value('tau_b'))
ratio = model.parameters.get_value('tau_a') / model.parameters.get_value('tau_b')

# Assume initial state is depleted (e.g., 50% of equilibrium)
f_B_0 = 0.5 * model.parameters.get_value('f_B_eq')
f_B_recovery = model.parameters.get_value('f_B_eq') + (f_B_0 - model.parameters.get_value('f_B_eq')) * jnp.exp(-time_pred / tau_eq)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Bridge fraction recovery
ax1.semilogx(time_pred, f_B_recovery, '-', linewidth=2)
ax1.axhline(model.parameters.get_value('f_B_eq'), color='r', linestyle='--', alpha=0.5, label=f'f_B_eq = {model.parameters.get_value('f_B_eq'):.4f}')
ax1.axhline(f_B_0, color='g', linestyle='--', alpha=0.5, label=f'f_B_0 = {f_B_0:.4f}')
ax1.axvline(tau_eq, color='purple', linestyle='--', alpha=0.5, label=f'τ_eq = {tau_eq:.4e} s')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Bridge Fraction f_B', fontsize=12)
ax1.set_title('Bridge Fraction Recovery', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Effective modulus evolution
G_eff_recovery = f_B_recovery * model.parameters.get_value('G') * jnp.exp(-time_pred / model.parameters.get_value('tau_b'))
ax2.loglog(time_pred, G_eff_recovery, '-', linewidth=2, label='G(t) with recovery')
ax2.loglog(time_pred, model.parameters.get_value('G') * model.parameters.get_value('f_B_eq') * jnp.exp(-time_pred / model.parameters.get_value('tau_b')), '--', 
           linewidth=2, alpha=0.5, label='G(t) no recovery')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('G(t) (Pa)', fontsize=12)
ax2.set_title('Modulus Evolution with Re-equilibration', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")
plt.close('all')

print(f"\nBridge re-equilibration:")
print(f"  Equilibrium bridge fraction: {model.parameters.get_value('f_B_eq'):.4f}")
print(f"  Re-equilibration time tau_eq: {tau_eq:.4e} s")
print(f"  tau_a / tau_b ratio: {ratio:.4f}")

## Physical Analysis: Two-Timescale Dynamics

In [ ]:
# Decompose relaxation into fast (elastic) and slow (structural) components
G_elastic = model.parameters.get_value('G') * model.parameters.get_value('f_B_eq') * jnp.exp(-time_pred / model.parameters.get_value('tau_b'))
G_total = modulus_pred

fig, ax = plt.subplots(figsize=(10, 7))
ax.loglog(time_pred, G_total, '-', linewidth=2, label='Total G(t)', color='blue')
ax.loglog(time_pred, G_elastic, '--', linewidth=2, label=f'Elastic decay (τ_b = {model.parameters.get_value('tau_b'):.2e} s)', color='red', alpha=0.7)
ax.axvline(model.parameters.get_value('tau_b'), color='red', linestyle=':', alpha=0.5, label='τ_b')
ax.axvline(tau_eq, color='purple', linestyle=':', alpha=0.5, label='τ_eq')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('G(t) (Pa)', fontsize=12)
ax.set_title('Two-Timescale Relaxation', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close("all")
plt.close('all')

print(f"\nTimescale separation:")
print(f"  Fast (elastic) decay: tau_b = {model.parameters.get_value('tau_b'):.4e} s")
print(f"  Slow (structural) recovery: tau_a = {model.parameters.get_value('tau_a'):.4e} s")
print(f"  Effective recovery time: tau_eq = {tau_eq:.4e} s")
print(f"  Separation factor: tau_a/tau_b = {ratio:.4f}")

if ratio > 10:
    print(f"  → Well-separated timescales: fast elastic relaxation, slow structural recovery")
elif ratio > 2:
    print(f"  → Moderate separation: both processes observable")
else:
    print(f"  → Coupled relaxation: single effective timescale")

## Physical Analysis: Ratio Effects

In [ ]:
# Sweep tau_a/tau_b ratio to show relaxation variation
ratio_sweep = jnp.array([0.1, 1.0, 10.0, 100.0])
time_sweep = jnp.logspace(-3, 2, 200)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

for r in ratio_sweep:
    tau_a_sweep = r * model.parameters.get_value('tau_b')
    f_B_eq_sweep = 1.0 / (1.0 + r)
    tau_eq_sweep = tau_a_sweep * model.parameters.get_value('tau_b') / (tau_a_sweep + model.parameters.get_value('tau_b'))
    f_B_sweep = f_B_eq_sweep + (f_B_0 - f_B_eq_sweep) * jnp.exp(-time_sweep / tau_eq_sweep)
    G_sweep = f_B_sweep * model.parameters.get_value('G') * jnp.exp(-time_sweep / model.parameters.get_value('tau_b'))
    
    ax1.loglog(time_sweep, G_sweep, '-', linewidth=2, label=f'τ_a/τ_b = {r:.1f}')

ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('G(t) (Pa)', fontsize=12)
ax1.set_title('Relaxation Modulus vs Ratio', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Equilibrium bridge fraction vs ratio
ratio_range = jnp.logspace(-1, 2, 50)
f_B_eq_range = 1.0 / (1.0 + ratio_range)
ax2.semilogx(ratio_range, f_B_eq_range, '-', linewidth=2)
ax2.axhline(0.5, color='r', linestyle='--', alpha=0.3)
ax2.axvline(1.0, color='r', linestyle='--', alpha=0.3)
ax2.set_xlabel('τ_a / τ_b', fontsize=12)
ax2.set_ylabel('Equilibrium f_B', fontsize=12)
ax2.set_title('Bridge Fraction vs Kinetic Ratio', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1])

plt.tight_layout()
plt.close("all")
plt.close('all')

## Bayesian Inference

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayes_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayes_result = model.fit_bayesian(
        time_data, modulus,
        test_mode='relaxation',
        gamma_dot=gamma_dot,
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(bayes_result, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## Parameter Comparison: NLSQ vs Bayesian

In [ ]:
print_parameter_comparison(model, bayes_result.posterior_samples, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayes_result, 'to_inference_data'):
    display_arviz_diagnostics(bayes_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayes_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        time_data,
        G_t,
        model, bayes_result, test_mode="relaxation",
        param_names=param_names, log_scale=True,
        xlabel=r'Time (s)',
        ylabel=r'Relaxation modulus $G(t)$ (Pa)', gamma_dot=gamma_dot
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation

In [ ]:
print("\n=== Physical Interpretation ===")
print(f"\n1. Material Properties:")
print(f"   - Plateau modulus G: {model.parameters.get_value('G'):.4e} Pa")
print(f"   - Equilibrium bridge fraction: {model.parameters.get_value('f_B_eq'):.4f}")
print(f"   - Equilibrium modulus: {model.parameters.get_value('G') * model.parameters.get_value('f_B_eq'):.4e} Pa")

print(f"\n2. Relaxation Timescales:")
print(f"   - Bridge detachment time tau_b: {model.parameters.get_value('tau_b'):.4e} s")
print(f"   - Loop attachment time tau_a: {model.parameters.get_value('tau_a'):.4e} s")
print(f"   - Re-equilibration time tau_eq: {tau_eq:.4e} s")
print(f"   - Ratio tau_a/tau_b: {ratio:.4f}")

print(f"\n3. Two-Timescale Behavior:")
if ratio > 10:
    print(f"   - Well-separated timescales detected")
    print(f"   - Fast elastic relaxation (~ tau_b) followed by slow structural recovery (~ tau_a)")
elif ratio > 2:
    print(f"   - Moderate timescale separation")
    print(f"   - Both elastic and structural processes observable")
else:
    print(f"   - Coupled relaxation-recovery")
    print(f"   - Single effective timescale (~ tau_eq)")

print(f"\n4. Bridge Re-equilibration:")
print(f"   - Initial depletion assumed: 50% of f_B_eq")
print(f"   - Recovery follows: f_B(t) = f_B_eq + (f_B_0 - f_B_eq) * exp(-t/tau_eq)")
print(f"   - Time to 90% recovery: {-tau_eq * jnp.log(0.1):.4e} s")

print(f"\n5. Modulus Decay:")
initial_modulus = modulus_pred[0]
final_modulus = modulus_pred[-1]
decay_ratio = final_modulus / initial_modulus
print(f"   - Initial modulus: {initial_modulus:.4e} Pa")
print(f"   - Final modulus: {final_modulus:.4e} Pa")
print(f"   - Decay ratio: {decay_ratio:.4f}")
print(f"   - Decades of relaxation: {jnp.log10(initial_modulus/final_modulus):.2f}")

## Save Results

In [ ]:
save_tnt_results(model, bayes_result, "loop_bridge", "relaxation", param_names)
print("Results saved to reference_outputs/tnt/loop_bridge_relaxation_results.npz")

## Key Takeaways

1. **Two-Timescale Relaxation**: Fast elastic decay (tau_b) and slow structural recovery (tau_a)

2. **Bridge Re-equilibration**: f_B recovers exponentially with time constant tau_eq = tau_a * tau_b / (tau_a + tau_b)

3. **Ratio Control**: tau_a/tau_b determines equilibrium bridge fraction and recovery rate

4. **Physical Mechanism**: Detachment (loops ← bridges) competes with attachment (loops → bridges)

5. **Timescale Separation**: High ratio → well-separated processes, low ratio → coupled dynamics

6. **Model Limitation**: Assumes no aging or thixotropic effects during relaxation

7. **Experimental Design**: Relaxation after pre-shear reveals both tau_b and tau_a independently

## Next Steps

- **Notebook 16**: Continue exploring this model family
- **Advanced Models**: Compare with other TNT variants (Notebooks 07-30)


## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
